In [1]:
'''
Created on Jun 24, 2018

@author: Mark Holton
'''

import sys
import pandas as pd
import numpy as np
import qiime2

from qiime2 import Metadata
from collections import defaultdict


## PEP8 all code 

it's bad practice to assume someone is going to give you a filename without an extension.  
it there a particular reason why you want to have a function that basically only does `readlines` method?  
Why not do all of the parsing of `user_input_file_of_queries` in one function, i.e. provide a filename and output a formatted data structure to do downstream operations?

In [2]:
def get_user_input_query_lines(user_input_file_of_queries):
    '''
    converts user_input_file_of_queries file into a list of strings that represent the lines of the file

    Parameters
    ----------
    user_input_file_of_queries : string
        file that contains lines of stings  
    
    Returns
    -------
    lines : list 
        list of strings that are the lines of the file
    '''
    return open('./%s'%(user_input_file_of_queries),'r').readlines()

## my take on `keep_samples`

In [3]:
def keep_samples(original_MD, keep_query_lines):
    '''
    Filters out unwanted rows based on values in chosen columns.
    
    Parameters
    ----------
    original_MD : Metadata object
        Metadata object with all samples 
    
    keep_query_lines : array of strings
        list of strings that are the lines of the file
        each string is a sqlite query that determines what ids to keep

    Returns
    -------
    shrunk_MD : Metadata object
        original_MD input except that desired exclution has been applied so only the samples that match the input querys
        are kept
    '''
    shrunk_MD = original_MD
    shrunk_MD = shrunk_MD.filter_ids(shrunk_MD.get_ids(' AND '.join(keep_query_lines)))
        
    return shrunk_MD
    

try to simplify `get_case_controlDF` function and consider renaming it - providing data type in the name in not common

In [4]:
def determine_cases_and_controls(afterExclusion_MD, query_line_dict, case_controlDF):
    '''
    Determines what samples are cases or controls using the queries in query_line_array. The labels of each sample are 
    stored in case_controlDF
    
    Parameters
    ----------
    afterExclusion_MD : Metadata object
        Metadata object with unwanted samples filtered out
        
    query_line_array : array of 2 arrays of strings
        the arrays of strings are arrays of queries
        the first array are make of queries to determine controls
        the second array are make of queries to determine cases

        
    case_controlDF : dataframe
        dataframe with one column named case_control. The indexs are the same as the indexs of afterExclution_MD
        all values are Undefined
        
    Returns
    -------
    case_controlDF : dataframe
        dataframe with one column named case_control. The indexs are the same as the indexs of afterExclution_MD  
        values reflect if the index is a case, control, or Undefined
    '''
    afterExclusion_MD_full = afterExclusion_MD
    #change case_or_control to reflect that the next loop of queries in query_lines will filter based on control queries
   
    for key in query_line_dict:
        if key != 'case' and key != 'control':
            continue
        #resets afterExclution_MD so that filtering down to control samples does not influence filtering down to case
        shrunk_MD = afterExclusion_MD_full
            
        query_lines = query_line_dict[key]
        shrunk_MD = shrunk_MD.filter_ids(shrunk_MD.get_ids(' AND '.join(query_lines)))
        ids = shrunk_MD.ids
        
        #replaces the true values created by the loop above to case or control
        case_controlDF.loc[ids,'case_control'] = key

    
    return case_controlDF

In [5]:
def merge_case_controlDF_and_afterExclutionMD(afterExclusion_MD, case_controlDF):
    '''
    Combines case_controlDF with afterExclution_MD and returns it as a metadata object
    
    Parameters
    ----------
    afterExclusion_MD : Metadata object
        Metadata object with unwanted samples filtered out
        
    case_controlDF : dataframe
        dataframe with one column named case_control. The indexs are the same as the indexs of afterExclution_MD  
        values reflect if the index is a case, control, or Undefined
        
    Returns
    -------
    Metadata(returnedMD) : Metadata object
        Metadata object with unwanted samples filtered out and a case_control column that reflects if the index is 
        a case, control, or Undefined    
    '''
    #turns case_controlDF into a metadata object
    case_controlMD = Metadata( case_controlDF)
    index_header_name = afterExclusion_MD.id_header
    #merges afterExclution_MD and case_controlMD into one new metadata object
    mergedMD = Metadata.merge(afterExclusion_MD, case_controlMD)

    return mergedMD


In [6]:
def filter_prep_for_matchMD(merged_MD, match_condition_lines, null_value_lines):
    '''
    filters out samples that do not have valid entries for columns that determine matching
    
    Parameters
    ----------
    merged_MD : Metadata object
        has case_control with correct labels but some samples might not have all matching information
    
    match_condition_lines : array of strings
        contains conditons to match samples on. In this function it is used only to get the columns for matching.

    Returns
    -------
    returned_MD : Metadata object
        Samples that do not have valid entries for columns that determine matching are removed. Everything else is the
        same as merged_MD.
    '''
    returned_MD = merged_MD
    for condition in match_condition_lines:
        column = condition.split('\t')[1]
        ids = returned_MD.get_ids(column + ' NOT IN ' + null_value_lines[0])
        #shrinks the MD so that future ids do not include samples that fail past queries
        merged_MD = returned_MD.filter_ids(ids)



    return merged_MD


In [7]:
def orderDict(dictionary, value_frequency):
    '''
    orders the elements of each array that is associated with a sample key by how often they get matched to other samples
    least to greatest
    
    Parameters
    dictionary: dictionary
        keys are linked to arrays that contain strings that correspond to samples that match to the
        sample the key represents
        
    value_frequency: dictionary
        keys are samples found in arrays of dictionary elements are numerical representation of 
        how many samples it matches to 
    
    Returns
    dictionary_to_return: dictionary
        dictionary with elements of the arrays that correspond to keys ordered from least to greatest abundance
    '''
    dictionary_to_return = dictionary.copy()
    for key in dictionary_to_return:
        unordered_array = dictionary_to_return[key]
        if len(unordered_array) == 0:
            continue
        #ordered_array will store the elements linked to key in their proper order    
        ordered_array = []
        count = 0
        while count < len(unordered_array):
            value = unordered_array[count]
            index = 0 #this index is for going through when sorting
            while index < len(ordered_array):
                ordered_value = ordered_array[index]
                if value_frequency[ordered_value] > value_frequency[value]:
                    ordered_array.insert(index,value)
                    break
                index = index + 1
            if index == len(ordered_array):
                ordered_array.insert(index,value)
                
            count = count + 1
        #overrides the unordered array linked to key with its ordered array
        dictionary_to_return[key] = ordered_array

    return dictionary_to_return
                

In [8]:
def order_keys(dictionary):
    '''
    orders the keys of a dictionary so that they can be used properly as the freemen of stable marriage 

    Parameters
    dictionary
        dictionary of cases or controls with their matching controls or cases ordered 
        by rising abundance
    
    Return
    keys_least_to_greatest: list
        contains keys in order of least to greatest amount of samples they match to
    '''
    keys_least_to_greatest = []
    for key in dictionary:
        if keys_least_to_greatest == []:
            keys_least_to_greatest.append(key)
            continue
        abundance_of_key_values = len(dictionary[key])
        index = 0
        for list_key in keys_least_to_greatest:
            abundance_of_list_key_values = len(dictionary[list_key])
            if abundance_of_key_values < abundance_of_list_key_values:
                keys_least_to_greatest.insert(index, key)
                break
            index = index + 1
        if index == len(keys_least_to_greatest):
            keys_least_to_greatest.append(key)

    return keys_least_to_greatest

In [9]:
def stable_marriage(dictionary_pref, pref_counts):
    '''
    based on code shown by Tyler Moore in his slides for Lecture 2 for CSE 3353, SMU, Dallas, TX
    these slides can be found at https://tylermoore.ens.utulsa.edu/courses/cse3353/slides/l02-handout.pdf
    
    Gets back the best way to match samples to eachother to in a one to one manner. Best way refers to getting back the 
    most amount of one to one matches. 
    Note:
        If the keys of dictionary_pref are case samples the keys of one_to_one_match_dictionary will be control samples
    
    Parameters
    dictionary_pref: dictionary
        dictionary_pref is a dictionary of cases or controls with their matching controls or cases ordered 
        by rising abundance
    
    pref_counts: dictionary
        pref_counts is a dictionary with the frequency controls or cases match to something in dictionary_pref
    
    Returns
    one_to_one_match_dictionary: dictionary
        dictionary with keys and their corresponding values representing a match between a case and control sample
    '''
  
    free_keys = order_keys(dictionary_pref)
    #print(free_keys)
    one_to_one_match_dictionary = {}
    while free_keys :
        key = free_keys.pop()
        if len(dictionary_pref[key]) == 0:
            continue
        #get the highest ranked woman that has not yet been proposed to
        entry = dictionary_pref[key].pop()
        if entry not in one_to_one_match_dictionary: 
            one_to_one_match_dictionary[entry] = key
        else:
            key_in_use = one_to_one_match_dictionary [entry]
            if pref_counts[key] < pref_counts[key_in_use]:
                one_to_one_match_dictionary[entry] = key
                free_keys.append(key_in_use)
            else:
                free_keys.append(key)

    return one_to_one_match_dictionary

In [10]:
def match_samples(prepped_for_match_MD, conditions_for_match_lines):
    '''
    matches case samples to controls and puts the case's id in column matched to on the control sample's row
    
    Parameters
    ----------
    prepped_for_match_MD : Metadata object
        Samples that do not have valid entries for columns that determine matching are removed. Everything else is the
        same as merged_MD.
    
    conditions_for_match_lines : dataframe
        contains information on what conditions must be met to constitue a match
    
    Returns
    -------
    masterDF : dataframe
        masterDF with matches represented by a column called matched to. Values in matched to are sample id of the case
        sample the control sample matches to or 1 if it is a case sample
    
    '''
    case_dictionary = {}
    control_dictionary = {}
    control_match_count_dictionary = {}
    case_match_count_dictionary = {}
    
    matchDF = prepped_for_match_MD.to_dataframe()
    case_for_matchDF = matchDF[matchDF['case_control'].isin(['case'])]
    # creates column to show matches. since it will contain the sample number it was matched too the null value will be 0
    matchDF['matched_to'] = '0'

    

    # loops though case samples and matches them to controls
    for case_index, case_row in case_for_matchDF.iterrows():
        #print('case index is %s'%(case_index))
        
        # set matchDF to be only the samples of masterDF that are control samples
        controlDF = matchDF.copy()
        controlDF = controlDF[controlDF['case_control'].isin(['control'])]

        # loop though input columns to determine matches
        for conditions in conditions_for_match_lines:
            
            column_name = conditions.split('\t')[1].strip()
            
            # get the type of data for the given column. This determine how a match is determined
            if conditions.split('\t')[0] == 'range':
                num = conditions.split('\t')[2]
                # filters controls based on if the value in the control is not within a given distance form the case
                controlDF = controlDF[
                                    ( pd.to_numeric(controlDF[column_name]) >= ( int(float(case_row[column_name])) - int(num) ) ) 
                                    &
                                    ( pd.to_numeric(controlDF[column_name]) <= ( int(float(case_row[column_name])) + int(num) ) )
                                   ] 
            else:
                # filters controls if the strings for the control and case don't match
                controlDF = controlDF[controlDF[column_name].isin([case_row[column_name]])]
        case_dictionary.update({case_index:controlDF.index.values})
            
        # sets the matched to column of masterDF to the case sample ID for the control samples still left in matchDF
        #sets the control sample 'matched to' value to sample id in master_frame which is the same as the case's index
        #matchDF['matched_to'] = np.where(matchDF.index.isin(controlDF.index), matchDF['matched_to'] +' ' + case_index, matchDF['matched_to'] )
        
        for id_control in controlDF.index:
            if id_control not in control_match_count_dictionary:
                control_match_count_dictionary.update({id_control:0})
            control_match_count_dictionary.update({id_control:(control_match_count_dictionary[id_control]+1)})
            
            '''if id_control not in control_dictionary:
                control_dictionary.update({id_control:[case_index]})
            else:
                control_dictionary[id_control].append(case_index)'''
        '''if case_index not in case_match_count_dictionary:
            case_match_count_dictionary.update({case_index:0})'''
        case_match_count_dictionary.update({case_index:(controlDF.index.values.size)})
        

        
    '''control_orig = control_dictionary.copy()
    control_dictionary = orderDict(control_dictionary, case_match_count_dictionary)
    control_to_case_match = stable_marriage(control_dictionary.copy(), control_match_count_dictionary)
    control_dictionary = control_orig'''
    
    case_orig = case_dictionary.copy()
    case_dictionary = orderDict(case_dictionary, control_match_count_dictionary)
    case_to_control_match = stable_marriage(case_dictionary.copy(), case_match_count_dictionary)
    case_dictionary = case_orig
    

    for key in case_to_control_match:
        key_value = case_to_control_match[key]
        matchDF.loc[ key, 'matched_to'] = key_value
        matchDF.loc[ key_value, 'matched_to'] = key

    
    return Metadata(matchDF) 

In [11]:
#metadata file
file_of_metadata = 'psych_bins_saliva.txt' 
user_input_file_name_exclude = 'psych_bins_saliva filter.txt'
user_input_file_name_control = 'psych_bins_saliva control.txt'
user_input_file_name_experiment = 'psych_bins_saliva case.txt'
user_input_file_null_values = 'psych_bins_saliva_null_values.txt'
user_input_file_name_match = 'psych_bins_saliva match.txt' 

In [12]:
#each line is a sqlite query to determine what samples to keep
exclude_query_lines_input = get_user_input_query_lines(user_input_file_name_exclude)
#each line is a sqlite query to determine what samples to label control
control_query_lines_input = get_user_input_query_lines(user_input_file_name_control)
#each line is a sqlite query to determine what samples to label case
case_query_lines_input = get_user_input_query_lines(user_input_file_name_experiment)
null_values_lines_input = get_user_input_query_lines(user_input_file_null_values)

case_control_dict = {'case':case_query_lines_input, 'control':control_query_lines_input }

'''
each line is tab seperated
the first element is the type of match: range or exact
    range matches samples if the numerical values compared are with in some other number of eachother
        this is only to be used with numerical values
    exact matches samples if the values compared are exactly the same
        this can be used for strings and numbers
the second element is the column to compare values of for the case and control samples
the third element is the range number or = (if the match type is exact) 
    this determines how far away a sample can be from another sample for the given column to be matched
'''
match_condition_lines_input = get_user_input_query_lines(user_input_file_name_match)


MD stands for Metadata. Metadata objects will have MD at the end of it. DF stands for dataframe. Dataframe objects end with DF.

In [13]:
#read metadata file into metadata object
originalMD = Metadata.load( file_of_metadata )
#originalMD.to_dataframe()

In [14]:
afterExclusionMD = keep_samples(originalMD, exclude_query_lines_input)
#afterExclusionMD.to_dataframe()

Lables the samples based on if they are control or not

In [15]:
ids = afterExclusionMD.get_ids()
case_control_Series = pd.Series( ['Unspecified'] * len(ids), ids)
'''
['Unspecified'] * len(ids) creates a list of elements. The list is the 
same length as ids. All the elements are 'Unspecified'
'''
case_control_Series.index.name = afterExclusionMD.id_header
case_controlDF = case_control_Series.to_frame('case_control') 
#case_controlDF

In [16]:
case_controlDF = determine_cases_and_controls(afterExclusionMD, case_control_dict, case_controlDF)
#case_controlDF

In [17]:
case_controlMD = merge_case_controlDF_and_afterExclutionMD(afterExclusionMD, case_controlDF)
#case_controlMD.to_dataframe()

Filters out data with match columns left unspecified

In [18]:
prepped_for_matchMD= filter_prep_for_matchMD(case_controlMD, match_condition_lines_input, null_values_lines_input)
#prepped_for_matchMD.to_dataframe()

Matches samples labled experiment to control samples

In [19]:
matchedMD = match_samples( prepped_for_matchMD, match_condition_lines_input )
matchedDF = matchedMD.to_dataframe()
matchedDF

,age,age_units,anonymized_name,antibiotic_indication,antibiotic_type,antibiotics_last_6_mo,any_health_condition_that_may_affect_microbiome,any_health_condition_that_may_affect_microbiome_describe,appendix_removed,are_you_aware_of_other_study_participants,...,ia_interovextero_insula_l,ia_interovextero_insula_r,ia_interovextero_insula_avg,ia_interovextero_insula_avg_norm,OASIS_bin,DAST_bin,PHQ_bin,SCOFF_bin,case_control,matched_to
id,,,,,,,,,,,,,,,,,,,,,
10424.AA181.O,22.0,years,AA181,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,-0.000329,0.000546,0.000108,0.006860,A,A,A,A,control,10424.AT668.O
10424.AA512.O,44.0,years,AA512,not applicable,not applicable,No,No,not applicable,Yes,missing: not provided,...,0.003893,0.002637,0.003265,0.206443,A,A,A,A,control,10424.AS863.O
10424.AA576.O,46.0,years,AA576,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,0.000838,0.001316,0.001077,0.068098,A,A,A,A,control,10424.AQ031.O
10424.AA624.O,42.0,years,AA624,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,-0.000404,-0.000875,-0.000639,-0.040435,A,A,A,A,control,10424.AN442.O
10424.AB765.O,20.0,years,AB765,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,0.000127,0.002662,0.001395,0.088173,A,A,A,A,control,10424.AM856.O
10424.AD421.O,28.0,years,AD421,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,0.001826,0.001147,0.001486,0.093990,A,A,A,A,control,10424.AS009.O
10424.AD656.O,52.0,years,AD656,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,0.003950,0.001339,0.002645,0.167209,A,A,A,A,control,10424.AM257.O
10424.AD710.O,23.0,years,AD710,not applicable,not applicable,No,Yes,eczema but not active,No,missing: not provided,...,0.001991,0.002258,0.002125,0.134330,A,A,A,A,control,0
10424.AF385.O,21.0,years,AF385,not applicable,not applicable,No,No,not applicable,No,missing: not provided,...,0.003392,0.004351,0.003872,0.244792,A,A,A,A,control,0


In [20]:
control_keys = matchedDF[(matchedDF['matched_to'] != '0')]['case_control'].index
control_keys

Index(['10424.AA181.O', '10424.AA512.O', '10424.AA576.O', '10424.AA624.O',
       '10424.AB765.O', '10424.AD421.O', '10424.AD656.O', '10424.AF655.O',
       '10424.AG513.O', '10424.AH589.O', '10424.AH917.O', '10424.AI548.O',
       '10424.AI557.O', '10424.AI961.O', '10424.AJ645.O', '10424.AJ695.O',
       '10424.AJ757.O', '10424.AJ813.O', '10424.AK245.O', '10424.AK263.O',
       '10424.AK271.O', '10424.AK314.O', '10424.AK322.O', '10424.AK407.O',
       '10424.AK523.O', '10424.AK662.O', '10424.AK714.O', '10424.AL016.O',
       '10424.AL323.O', '10424.AL404.O', '10424.AL417.O', '10424.AL517.O',
       '10424.AL731.O', '10424.AL906.O', '10424.AL939.O', '10424.AM040.O',
       '10424.AM067.O', '10424.AM141.O', '10424.AM152.O', '10424.AM257.O',
       '10424.AM334.O', '10424.AM395.O', '10424.AM663.O', '10424.AM856.O',
       '10424.AN442.O', '10424.AN574.O', '10424.AN590.O', '10424.AN861.O',
       '10424.AO025.O', '10424.AO032.O', '10424.AO549.O', '10424.AO593.O',
       '10424.AO610.O', '